<a href="https://colab.research.google.com/github/lupis30puc/yelp_bert_random_forest/blob/lupis30puc-update-2/Yelp_RF_mimic_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Yelp Polarity on kaggle](https://www.kaggle.com/yelp-dataset/yelp-dataset)

12,993 samples from the Yelp Dataset Challenge 2020. 
Divided on train, validation and test subsets. 
Their corresponding sizes are: 10,394 train samples, 1,949 validation samples and 650 test samples.


Tutorial on which I support: 
[Sentiment Analysis Yelp with Random Forest](https://www.kaggle.com/omkarsabnis/sentiment-analysis-on-the-yelp-reviews-dataset)

In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 10.6MB/s 
     |████████████████████████████████| 890kB 60.7MB/s 
     |████████████████████████████████| 2.9MB 36.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=4b205a255de41d0e982192a1e415444ac9694626a0253d53c234dc7928b9a5f4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
import string
import math
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
%matplotlib inline
import time

I want to create a boolean matrix with the input ids of the datasets...

In [4]:
import torch
test_dataset = torch.load('/content/drive/MyDrive/Yelp/sample_yelp_tensors/_test_150')
val_dataset = torch.load('/content/drive/MyDrive/Yelp/sample_yelp_tensors/_validation_150')
train_dataset = torch.load('/content/drive/MyDrive/Yelp/sample_yelp_tensors/_train_150')

In [5]:
len(train_dataset)

10394

In [6]:
len(test_dataset)

650

In [7]:
test_dataset[0][0].numpy()

array([  101,  2044,  3752,  2035,  1996,  2307,  4391,  2057,  2245,
        2057,  1040,  2507,  4012, 15455,  6072,  1037,  3046,  2005,
        6265,  2057,  4694,  1056,  9364,  2057,  2179,  1996, 25545,
        2465,  2100,  2007,  3737, 23127,  4406,  2070,  2060,  5171,
        4916,  7884,  2057,  2020, 11188, 22775,  1998,  3491,  2000,
        1037,  3332,  9065, 11772,  2048,  4127,  1997, 26509,  2020,
        5359, 13364,  1996, 12183,  2038,  2172,  2000,  5454,  2013,
        1045,  4900,  1996,  6265,  2569,  2007,  1037,  3601,  1997,
        2048,  3760,  4372, 13334,  2015,  1996, 26192,  2094,  7975,
        4372,  5428, 27266,  2050,  2007, 16709, 12901,  2001,  6581,
        2021,  2130,  2488,  2001,  1996,  7975,  9610,  7712, 18003,
        2050,  2009,  2001,  5642,  2011,  3903,  1997,  5785,  1998,
        2128, 13017, 13435,  2673,  2790,  4840,  1998, 25628,  1996,
        4825,  2003, 21121,  2011,  1045,  2903,  2155,  2372,  2013,
        3290,  2877,

In [8]:
# with this for loop I get the indexes, ids as arrays and labels into their correspodant lists for the next steps
test_index = []
test_ids = []
test_labels = []
for i in range(len(test_dataset)): 
    test_index.append(i)
    test_ids.append(test_dataset[i][0].numpy())
    test_labels.append(int(test_dataset[i][2]))
print(test_ids[2])
print(test_labels[2])

[  101  2200  4550  3309 14855 10841 13793  2003  2200  4121  6457  1998
  5723  2024  4121  1998  4550  3193  2001  3835  2348  2009  2001  1996
  2067  1997  1996  3309  9568  2001  3835  1998 27238  2001  2307  2085
  1996  2919  2694  2921 12809  2296  2261  2847  2005  2195  2781  2012
  1037  2051  2057  2020  2006  1996 16215  2723  1998  2657  1996  3345
  2296  2305  2035 27046  5833  1996  2305  5189  1998  4415 10047  1037
  8441  3345  3992  2005  1057  1052  1998  2008  2015  1996  2197  2518
  1045  2215 12447  2033  2039  2185  2013  2147  2061  2008  3084  6385
  1997  3714  3637  2035  2305  2196  2204  1045  2052  2025  2994  2182
  2153  2004  2057  2035  2113  1999  7136  2087  2111  2024  1999  2037
  3309  2069  2000  3637   102     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
0


In [9]:
print(len(test_index))

650


In [11]:
# this is to obtain the unique elements and their appearance in the array
a = test_ids[0]
unique_elements, counts_elements = np.unique(a, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[  101   102  1037  1040  1045  1056  1996  1997  1998  1999  2000  2001
   2003  2005  2007  2009  2011  2013  2015  2020  2021  2026  2035  2038
   2044  2045  2048  2050  2057  2060  2069  2070  2094  2100  2128  2130
   2155  2172  2179  2245  2256  2307  2344  2372  2465  2488  2507  2569
   2653  2673  2790  2877  2903  3046  3290  3325  3332  3491  3601  3737
   3752  3760  3903  4012  4127  4372  4391  4406  4694  4825  4840  4900
   4916  4997  5171  5359  5428  5454  5642  5785  6072  6265  6581  7263
   7615  7712  7884  7975  8803  9065  9364  9610 11188 11772 12183 12901
  13017 13334 13364 13435 15455 16709 18003 20888 21121 22775 23127 25545
  25628 26192 26509 27266]
 [    1     1     5     1     2     1     7     3     3     1     3     4
      1     1     3     1     2     2     1     2     1     1     3     1
      1     1     2     2     5     1     1     1     1     1     1     1
      1     1     1     1     1     1  

In [12]:
type(unique_elements)

numpy.ndarray

## Otros:

In [ ]:
# NO NEED FOR THE DATASET OR YES?
#train_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_train_10394.pkl')
#validation_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_validation_1949.pkl')
#test_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_test_650.pkl')

#sample_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_yelp_reviews_12993.pkl')

#train_df.reset_index(drop=True, inplace=True)
#validation_df.reset_index(drop=True, inplace=True)
#test_df.reset_index(drop=True, inplace=True)
#train_df.head()

In [13]:
# launching the saved model tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Yelp/model_save3/')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [15]:
tokenizer.vocab_size

30522

In [16]:
# after doing manual examination, obtaining the index of the last word in the vocab
%%time
for i in range(len(list(tokenizer.vocab.keys()))):
  if list(tokenizer.vocab.keys())[i] == 'necessitated':
    print(i)
    break

29611


In [ ]:
list(tokenizer.vocab.keys())[1996] #first actual word in vocab, got it through manual examination on the vocab text file...

'the'

In [ ]:
list(tokenizer.vocab.keys())[29611] #last actual word in vocab

'necessitated'

In [17]:
# 27615 words in total. (the other things on the vocab are symbols, numbers, etc.)
vocab = list(tokenizer.vocab.keys())[1996:29612]

## Continuación

In [18]:
# get a list of all unique ids
unique_ids = []
for i in test_index:
  uni_elements = np.unique(test_ids[i])
  for e in uni_elements: 
    if e not in unique_ids:
      unique_ids.append(e)

In [19]:
len(unique_ids)

5580

In [20]:
# making a dataframe where the index are the reviews index, and the columns are the unique words on the reviews.
df1 = pd.DataFrame(index=test_index, columns=unique_ids)
df1.fillna(0, inplace=True)
df1

,101,102,1037,1040,1045,1056,1996,1997,1998,1999,2000,2001,2003,2005,2007,2009,2011,2013,2015,2020,2021,2026,2035,2038,2044,2045,2048,2050,2057,2060,2069,2070,2094,2100,2128,2130,2155,2172,2179,2245,...,10169,17395,18971,19717,19905,5249,17331,4063,5978,6184,6313,6862,9313,10415,10672,16663,18906,5150,6498,19873,3888,14890,17460,25006,25249,17093,9963,24494,2400,6077,6380,12394,15047,18846,7946,8304,20014,24436,26986,18152
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
646,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
648,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
# running for loop to check fill in the dataframe when the word appears on the review
%%time
for i in test_index:
  for id in unique_ids:
    if id in test_ids[i]:
      df1[id][i] = 1

CPU times: user 23.2 s, sys: 41.7 ms, total: 23.2 s
Wall time: 23.3 s


In [24]:
if 10169  not in test_ids[0]:
  print(1)

1


In [23]:
df1.tail()

,101,102,1037,1040,1045,1056,1996,1997,1998,1999,2000,2001,2003,2005,2007,2009,2011,2013,2015,2020,2021,2026,2035,2038,2044,2045,2048,2050,2057,2060,2069,2070,2094,2100,2128,2130,2155,2172,2179,2245,...,10169,17395,18971,19717,19905,5249,17331,4063,5978,6184,6313,6862,9313,10415,10672,16663,18906,5150,6498,19873,3888,14890,17460,25006,25249,17093,9963,24494,2400,6077,6380,12394,15047,18846,7946,8304,20014,24436,26986,18152
645,1,1,1,0,1,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
646,1,1,1,0,1,1,0,0,1,0,0,0,0,1,1,1,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0
647,1,1,1,0,0,0,1,1,1,0,1,1,0,1,1,1,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0
648,1,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,0,0,1,1,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
649,1,1,1,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
df1.describe()

,101,102,1037,1040,1045,1056,1996,1997,1998,1999,2000,2001,2003,2005,2007,2009,2011,2013,2015,2020,2021,2026,2035,2038,2044,2045,2048,2050,2057,2060,2069,2070,2094,2100,2128,2130,2155,2172,2179,2245,...,10169,17395,18971,19717,19905,5249,17331,4063,5978,6184,6313,6862,9313,10415,10672,16663,18906,5150,6498,19873,3888,14890,17460,25006,25249,17093,9963,24494,2400,6077,6380,12394,15047,18846,7946,8304,20014,24436,26986,18152
count,650.0,650.0,650.000000,650.00000,650.000000,650.00000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,...,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000
mean,1.0,1.0,0.780000,0.04000,0.749231,0.38000,0.903077,0.592308,0.884615,0.561538,0.746154,0.621538,0.540000,0.590769,0.424615,0.613846,0.132308,0.190769,0.332308,0.315385,0.487692,0.429231,0.223077,0.109231,0.143077,0.256923,0.093846,0.035385,0.336923,0.113846,0.172308,0.138462,0.069231,0.121538,0.066154,0.116923,0.041538,0.101538,0.041538,0.030769,...,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.003077,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538,0.001538
std,0.0,0.0,0.414565,0.19611,0.433790,0.48576,0.296081,0.491784,0.319732,0.496581,0.435546,0.485377,0.498781,0.492071,0.494665,0.487242,0.339086,0.393210,0.471403,0.465027,0.500233,0.495348,0.416630,0.312169,0.350421,0.437273,0.291839,0.184892,0.473022,0.317869,0.377938,0.345650,0.254042,0.327004,0.248742,0.321576,0.199686,0.302273,0.199686,0.172825,...,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.055427,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223,0.039223
min,1.0,1.0,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,1.0,1.000000,0.00000,0.250000,0.00000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [87]:
print(unique_ids[:20])

[0, 101, 102, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053]


In [90]:
len(unique_ids)

5580

In [91]:
len(vocab)

27616

In [ ]:
vocab_test = []



## Training

In [26]:
# SPLITTING THE DATASET INTO TRAINING SET AND TESTING SET
# SAME RANDOM STATE AS IN BERT....
x_train,x_test,y_train,y_test = train_test_split(df1,test_labels,test_size=0.2,random_state=42)

In [27]:
%%time
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()

CPU times: user 59.7 ms, sys: 14.9 ms, total: 74.6 ms
Wall time: 284 ms


In [28]:
%%time
rmfr.fit(x_train,y_train)

CPU times: user 597 ms, sys: 5.02 ms, total: 602 ms
Wall time: 608 ms


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
%%time
predrmfr = rmfr.predict(x_test)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:")
print(classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[47 16]
 [16 51]]
Score: 75.38
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        63
           1       0.76      0.76      0.76        67

    accuracy                           0.75       130
   macro avg       0.75      0.75      0.75       130
weighted avg       0.75      0.75      0.75       130

CPU times: user 35.6 ms, sys: 3.01 ms, total: 38.6 ms
Wall time: 46.2 ms


In [30]:
importance = rmfr.feature_importances_

In [31]:
len(importance)

5580

In [118]:

feature_importance = pd.DataFrame({'keys': feature_names_test, 'imp': importance})

In [119]:
feature_importance.head()

,keys,imp
0,[PAD],0.000000
1,[CLS],0.000000
2,[SEP],0.002194
3,a,0.000632
4,b,0.002304


In [120]:
top20 = list(feature_importance.sort_values(by=['imp'], ascending=False)['keys'][:20])

In [121]:
# ran on monday jan 18
for word in top10:
  print(list(tokenizer.vocab.keys())[word])

great
not
then
delicious
definitely
to
good
order
got
no


In [ ]:
# ran on monday jan 11
for word in top10:
  print(list(tokenizer.vocab.keys())[word])

not
horrible
worst
delicious
don
good
asked
no
poor
definitely


In [36]:
!pip install treeinterpreter
#https://pypi.org/project/treeinterpreter/

In [37]:
from treeinterpreter import treeinterpreter as ti

In [38]:
print("Test  Accuracy : %.2f"%rmfr.score(x_test, y_test))
print("Train Accuracy : %.2f"%rmfr.score(x_train, y_train))

Test  Accuracy : 0.75
Train Accuracy : 1.00


In [94]:
preds, bias, contributions = ti.predict(rmfr, x_test)

In [95]:
preds.shape, bias.shape, contributions.shape

((130, 2), (130, 2), (130, 5580, 2))

In [110]:
# from http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/
print("Prediction", preds[0])
print("Bias (trainset prior)", bias[0])
print("Feature contributions:")
for c, feature in zip(contributions[0], top20):
  word = list(tokenizer.vocab.keys())[feature]
  print(word, c)

Prediction [0.37 0.63]
Bias (trainset prior) [0.50598077 0.49401923]
Feature contributions:
great [0. 0.]
not [0. 0.]
then [-0.00044358  0.00044358]
delicious [ 0.00032938 -0.00032938]
definitely [ 0.00169384 -0.00169384]
to [-0.00559167  0.00559167]
good [ 0.00351662 -0.00351662]
order [ 0.01040476 -0.01040476]
got [-0.00043478  0.00043478]
no [-0.00445752  0.00445752]
worst [ 0.00406524 -0.00406524]
that [ 0.00544223 -0.00544223]
horrible [ 0.00036786 -0.00036786]
amazing [ 0.0012525 -0.0012525]
friendly [-0.00357094  0.00357094]
nice [ 0.00438221 -0.00438221]
ordered [0. 0.]
asked [ 0.00018305 -0.00018305]
love [ 0.00088205 -0.00088205]
excellent [0. 0.]


In [100]:
print("Bias For Sample 0                        : %s"%bias[0])
print("Constributions For Sample 0              : %s"%contributions[0])
print("Prediction Based on Bias & Contributions : %s"%(bias[0] + contributions[0].sum()))
print("Actual Target Value                      : %s"%y_test[0])
print("Target Value As Per Treeinterpreter      : %s"%preds[0][0])

Bias For Sample 0                        : [0.50598077 0.49401923]
Constributions For Sample 0              : [[ 0.          0.        ]
 [ 0.          0.        ]
 [-0.00044358  0.00044358]
 ...
 [ 0.0002     -0.0002    ]
 [ 0.          0.        ]
 [ 0.          0.        ]]
Prediction Based on Bias & Contributions : [0.50598077 0.49401923]
Actual Target Value                      : 1
Target Value As Per Treeinterpreter      : 0.37


In [67]:
def create_contrbutions_df(contributions, random_sample, feature_names):
    contribs = contributions[random_sample].tolist()
    contribs.insert(0, bias[random_sample])
    contribs = np.array(contribs)
    contrib_df = pd.DataFrame(data=contribs, index=["Base"] + feature_names, columns=["Contributions_0", "Contributions_1"])
    prediction = contrib_df[["Contributions_0", "Contributions_1"]].sum()
    contrib_df.loc["Prediction"] = prediction
    return contrib_df

In [50]:
import random
random_sample = random.randint(1, len(x_test))
print("Selected Sample     : %d"%random_sample)
print("Actual Target Value : %s"%y_test[random_sample])
print("Predicted Value     : %s"%np.argmax(preds[random_sample]))

Selected Sample     : 103
Actual Target Value : 1
Predicted Value     : 1


In [116]:
# Converting unique_ids into words:
feature_names_test = []
for id in unique_ids:
  feature_names_test.append(list(tokenizer.vocab.keys())[id])

In [117]:
contrib_df = create_contrbutions_df(contributions, random_sample, feature_names_test)
contrib_df

# from: https://coderzcolumn.com/tutorials/machine-learning/treeinterpreter-interpreting-tree-based-models-prediction-of-individual-sample

,Contributions_0,Contributions_1
Base,0.505981,0.494019
[PAD],0.000000,0.000000
[CLS],0.000000,0.000000
[SEP],0.003013,-0.003013
a,-0.000709,0.000709
...,...,...
##pone,0.000000,0.000000
vo,0.000000,0.000000
infused,0.000000,0.000000
pudding,0.000000,0.000000


In [126]:
contrib_df.sort_values(by=['Contributions_0'], ascending=False)[:20]

,Contributions_0,Contributions_1
Base,0.505981,0.494019
Prediction,0.310000,0.690000
world,0.040212,-0.040212
that,0.025229,-0.025229
of,0.023962,-0.023962
than,0.012087,-0.012087
y,0.010056,-0.010056
shell,0.008491,-0.008491
##l,0.007970,-0.007970
lead,0.007819,-0.007819
